In [1]:
import asyncio
import websockets
import json
import logging
from jetbot import Robot, Camera, bgr8_to_jpeg
import base64
import time

# --- OpenCV Handling ---
try:
    import cv2
    OPENCV_AVAILABLE = True
except ImportError:
    cv2 = None
    OPENCV_AVAILABLE = False

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Configuration ---
WEBSOCKET_PORT = 8766
CAMERA_WIDTH = 224  # Reduced for faster processing
CAMERA_HEIGHT = 224  # Reduced for faster processing
FPS = 10  # Reduced to decrease network load
JPEG_QUALITY = 80  # Compression quality parameter
COMMAND_THROTTLE_TIME = 0.05  # Minimum time between command executions

# --- JetBot Initialization ---
try:
    robot = Robot()
    camera = Camera.instance(width=CAMERA_WIDTH, height=CAMERA_HEIGHT)
    logger.info("JetBot initialized successfully.")
except Exception as e:
    logger.error(f"JetBot initialization failed: {e}")
    robot = None
    camera = None

# --- Command Handling ---
last_command_time = 0
last_command = None
last_parameters = None

async def handle_command(command, parameters=None):
    global last_command_time, last_command, last_parameters
    
    # Throttle identical commands to prevent command flooding
    current_time = time.time()
    if (current_time - last_command_time < COMMAND_THROTTLE_TIME and 
        command == last_command and parameters == last_parameters):
        return
    
    last_command_time = current_time
    last_command = command
    last_parameters = parameters
    
    parameters = parameters or {}
    if not robot:
        logger.error("Robot not initialized.")
        return
    try:
        speed = parameters.get("speed", 0.4)
        steering = parameters.get("steering", 0.0)
        
        # Direct motor control for more precise movements
        if command == "forward":
            robot.set_motors(speed, speed)
        elif command == "backward":
            robot.set_motors(-speed, -speed)
        elif command == "left":
            robot.set_motors(-speed * 0.5, speed * 0.5)
        elif command == "right":
            robot.set_motors(speed * 0.5, -speed * 0.5)
        elif command == "stop":
            robot.stop()
        elif command == "u_turn":
            # Optimized U-turn implementation
            robot.set_motors(-speed, speed)
            await asyncio.sleep(1.0)  # Adjust time based on your robot's turning radius
            robot.stop()
        else:
            # Handle custom steering commands with direct motor control
            left_motor = speed * (1 - steering)
            right_motor = speed * (1 + steering)
            robot.set_motors(left_motor, right_motor)
            
    except Exception as e:
        logger.error(f"Command execution error: {e}")
        if robot:
            robot.stop()

# --- Image Processing ---
last_frame_time = 0
last_image_base64 = None

async def process_image():
    global last_frame_time, last_image_base64
    
    if not camera:
        return None
    
    current_time = time.time()
    # Limit frame rate
    if current_time - last_frame_time < 1/FPS:
        return last_image_base64
        
    try:
        frame = camera.value
        if frame is None:
            return None
            
        # 간단한 방식으로 이미지 인코딩 - 기본 bgr8_to_jpeg 사용
        image_base64 = base64.b64encode(bgr8_to_jpeg(frame)).decode('utf-8')
            
        last_frame_time = current_time
        last_image_base64 = image_base64
        return image_base64
    except Exception as e:
        logger.error(f"Image processing error: {str(e)}")
        return None

# --- WebSocket Handler ---
async def websocket_handler(websocket, path):
    logger.info("WebSocket connection established")

    async def send_image_stream():
        while True:
            try:
                image_base64 = await process_image()
                if image_base64:
                    # 디버깅을 위한 로그 추가
                    logger.debug(f"Sending image: {len(image_base64)} bytes")
                    await websocket.send(json.dumps({"image": image_base64}))
                await asyncio.sleep(1 / FPS)
            except Exception as e:
                logger.error(f"Image stream error: {str(e)}")
                await asyncio.sleep(1 / FPS)

    image_stream_task = asyncio.ensure_future(send_image_stream())
    try:
        async for message in websocket:
            try:
                data = json.loads(message)
                command = data.get("command", "none")
                parameters = data.get("parameters", {})
                # Process command in background to avoid blocking
                asyncio.create_task(handle_command(command, parameters))
            except json.JSONDecodeError:
                logger.error("Invalid JSON received")
    except websockets.exceptions.ConnectionClosed:
        logger.info("WebSocket connection closed")
    except Exception as e:
        logger.error(f"WebSocket error: {e}")
    finally:
        image_stream_task.cancel()
        if robot:
            robot.stop()

# --- Main Function ---
async def main():
    while True:
        try:
            server = await websockets.serve(
                websocket_handler, 
                "0.0.0.0", 
                WEBSOCKET_PORT,
                ping_interval=30,  # Keep connection alive
                ping_timeout=10,
                max_size=10_000_000  # 10MB 최대 메시지 크기 설정
            )
            logger.info(f"WebSocket server running on port {WEBSOCKET_PORT}")
            await asyncio.Future()
        except Exception as e:
            logger.error(f"Server error: {e}")
            await asyncio.sleep(5)

if __name__ == "__main__":
    if robot:
        robot.stop()
    asyncio.get_event_loop().run_until_complete(main())

2025-03-26 08:14:08,808 - INFO - JetBot initialized successfully.


RuntimeError: This event loop is already running